In [1]:
import numpy as np
from scipy.optimize import line_search

In [239]:
def logistic(x):
    return 1 / (1 + np.exp(-x))

def stable_logistic(x):
    """Logistic function (for a scalar valued argument).

    A numerically stable implementation.
    """
    return np.where(x > 0,
                    1 / (1 + np.exp(-x)),
                    np.exp(x) / (1 + np.exp(x)))

In [236]:
def phi(t):
    # logistic function, returns 1 / (1 + exp(-t))
#     idx = t &gt; 0
    out = np.empty(t.size, dtype=np.float)
    out = np.where(t > 0,
                  1. / (1 + np.exp(-t)),
                  np.exp(t) / (1. + np.exp(t)))
    return out


def loss(x0, X, y, alpha):
    # logistic loss function, returns Sum{-log(phi(t))}
    w, c = x0[:X.shape[1]], x0[-1]
    z = X.dot(w) + c
    yz = y * z
    out = np.zeros_like(yz)
    out = np.where(yz > 0, 
                   np.log(1 + np.exp(-yz)),
                  (-yz + np.log(1 + np.exp(yz))))
    out = out.sum()
    out /= X.shape[0] + 0.5*alpha*w.dot(w)
    return out

In [264]:
x = np.array([1, -1, 3, -6,0])
print(phi(x))
print(stable_logistic(x))
print(logistic(x))

[0.73105858 0.26894142 0.95257413 0.00247262 0.5       ]
[0.73105858 0.26894142 0.95257413 0.00247262 0.5       ]
[0.73105858 0.26894142 0.95257413 0.00247262 0.5       ]


In [266]:
1/ (1 + np.exp(-0))

0.5

In [322]:
def logistic_surrogate_loss(w, X, y):
    # Computing the dot product
    n, d = X.shape
    print(n)
    ps = np.dot(X, w[:-1]) + w[-1]
    print(ps.shape)
    yps = y * ps
#     loss = np.where(yps > 0,
#                    np.log(1 + np.exp(-yps)),
#                    (-yps + np.log(1 + np.exp(yps))))
#     loss = logistic(yps)
    loss = np.log(1. + np.exp(-yps))
    print(loss.shape)
#     loss = loss.sum()
#     loss /= n
    return np.mean(loss)

In [186]:
# def logistic_surrogate_loss_2(w, X, y):
#     n, d = X.shape
#     ps = np.dot(X, w[:-1]) + w[-1]
#     p = stable_logistic(-ps)
#     loss = (-y * np.log(p) - (1 - y) * np.log(1-p))
#     loss = loss.sum() / n
#     return loss

In [268]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=10, n_features=5)
w = np.random.random(X.shape[1]+1)

In [269]:
y = np.where(y == 0, -1, y)

In [270]:
print(logistic_surrogate_loss(w, X, y))
print(loss(w, X, y, 0))

0.30873441700392745
0.30873441700392745


In [245]:
# loss(w_ex, X_train, y_train, 0)

In [246]:
print(f"y shape: {y.shape}")
print(f"X shape is : {X.shape}")
print(f"w shape is : {w.shape}")

y shape: (10,)
X shape is : (10, 5)
w shape is : (6,)


In [247]:
(np.dot(X, w[1:]) + w[0]).shape

(10,)

In [248]:
def gradient(x0, X, y, alpha):
    # gradient of the logistic loss
    w, c = x0[:X.shape[1]], x0[-1]
    z = X.dot(w) + c
    z = phi(y * z)
    z0 = (z - 1) * y
    grad_w = X.T.dot(z0) / X.shape[0] # + alpha * w
    grad_c = z0.sum() / X.shape[0]
    return np.concatenate((grad_w, [grad_c]))

In [249]:
def gradient_log_surrogate_loss(w, X, y):
    # defining dim variables
    n, d = X.shape
    z = X.dot(w[:-1]) + w[-1]
#     z = stable_logistic(y * z)
    z = logistic(y*z)
    z0 = (z - 1) * y
    
    # initiating g: gradient vector
    g = np.zeros(d+1)
    # Computing dot product
    g[:-1] = X.T.dot(z0)
    g[-1] = z0.sum()
    g /= n
    return g


# line_search(test_func,test_grad,np.array([1.8,1.7]),np.array([-1.0,-1.0]))

In [250]:
def gradient_log_surrogate_loss_2(w, X, y):
    n, d = X.shape
    ps = np.dot(X, w[:-1]) + w[-1]
    ps = stable_logistic(y * ps)
    ps0 = (ps - 1) * y
    g = np.zeros(d+1)
    g[:-1] = np.dot(X.T, (ps - y)) / n
    g[-1] = ps0.sum()
    g /= n
    
    return g

In [251]:
print(gradient(w, X, y, 0))
print(gradient_log_surrogate_loss(w, X, y))
print(gradient_log_surrogate_loss_2(w, X, y))

[ 0.59929238  0.27318046  0.12409189  0.21932612 -0.26866456 -0.04335184]
[ 0.59929238  0.27318046  0.12409189  0.21932612 -0.26866456 -0.04335184]
[ 0.03394312  0.04658948  0.04306494  0.01859935 -0.08441464 -0.04335184]


In [252]:
g = gradient(w, X, y, 0.01)

In [253]:
g

array([ 0.59929238,  0.27318046,  0.12409189,  0.21932612, -0.26866456,
       -0.04335184])

In [254]:
line_search(f=loss, 
            myfprime=gradient,
           xk=w,pk=-g, gfk=g, old_fval=loss_val, args=(X, y, 0.1))

(1.0,
 1,
 1,
 0.4397278231618905,
 0.9495825573272951,
 array([ 0.31446402,  0.14625139,  0.12517708,  0.11566301, -0.14743893,
        -0.0255717 ]))

In [255]:
line_search(f=logistic_surrogate_loss, 
            myfprime=gradient_log_surrogate_loss,
           xk=w,pk=-g, gfk=g, old_fval=loss_val_2, args=(X, y))

(1.0,
 1,
 1,
 0.443708666499639,
 1.0497222245689655,
 array([ 0.31446402,  0.14625139,  0.12517708,  0.11566301, -0.14743893,
        -0.0255717 ]))

In [256]:
from scipy.optimize import fmin_cg
fmin_cg(f=loss, x0=w, fprime=gradient, args=(X, y, 0.01))

Optimization terminated successfully.
         Current function value: 0.000004
         Iterations: 26
         Function evaluations: 114
         Gradient evaluations: 114


array([ -6.0679659 , -59.14261181, -56.98487358, -12.93791546,
       128.80014714,   1.65235315])

In [257]:
from scipy.optimize import fmin_cg
fmin_cg(f=logistic_surrogate_loss, x0=w, fprime=gradient_log_surrogate_loss, args=(X, y))

Optimization terminated successfully.
         Current function value: 0.000009
         Iterations: 13
         Function evaluations: 60
         Gradient evaluations: 60


array([ 20.4359244 , -31.84404716, -15.44250819,  -0.21741124,
        83.07861191,  49.26389559])

In [168]:
def conjugate_gradient(w, cost_func, grd_func, args=(), epsilon=1.49e-08):
    
    n,d = X.shape
    w_old = np.random.random(d+1)
    
    new_loss = cost_func(w_old, *args)
    old_loss = new_loss + 2 * epsilon
    g = grd_func(w, *args)
    
    # Initialisation du de p au gradient en temps 0 (Eq 2.46)
    p = -g

    epoque = 0
    while np.abs(old_loss - new_loss) > np.abs(old_loss) * epsilon:
        old_loss = new_loss
        alpha, fc, gc, new_loss, old_loss, new_slope = line_search(f=cost_func, 
                                                                   myfprime=grd_func,
                                                                   xk=w,pk=p, gfk=g, old_fval=old_loss, 
                                                                   args=args)
        w = w + alpha * p        
        h = grd_func(w, *args)
        dgg = g @ g
        ngg = h @ h
        
        # Formule de Fletcher-Reeves (Eq 2.53)
        beta = ngg / dgg
        
        w_old = w
        g = h
        p = -g + beta * p
        print(f"Epoque {epoque} and loss is: {new_loss}")
        epoque += 1

In [234]:
# X, y = make_classification(n_samples=10, n_features=5)
# w = np.random.random(X.shape[1]+1)
conjugate_gradient(w, logistic_surrogate_loss, gradient_log_surrogate_loss, args=(X, y))

Epoque 0 and loss is: 0.5030240520009269
Epoque 1 and loss is: 0.39537658954965454
Epoque 2 and loss is: 0.34002837693990273
Epoque 3 and loss is: 0.303100974585575
Epoque 4 and loss is: 0.2754635701144111
Epoque 5 and loss is: 0.2533546917770667
Epoque 6 and loss is: 0.2369454927808444
Epoque 7 and loss is: 0.22524704029420542
Epoque 8 and loss is: 0.21589713235408298
Epoque 9 and loss is: 0.20709329630450474
Epoque 10 and loss is: 0.19819627545068744
Epoque 11 and loss is: 0.1895357122213285
Epoque 12 and loss is: 0.17452221942830937
Epoque 13 and loss is: 0.16295635957831756
Epoque 14 and loss is: 0.15233712747979725
Epoque 15 and loss is: 0.14393921295120182
Epoque 16 and loss is: 0.1377176182853582
Epoque 17 and loss is: 0.12583244707650448
Epoque 18 and loss is: 0.11145473039044174
Epoque 19 and loss is: 0.10191393645796794
Epoque 20 and loss is: 0.08909031652159541
Epoque 21 and loss is: 0.08329818246738623
Epoque 22 and loss is: 0.07535438433898714
Epoque 23 and loss is: 0.0634

/home/iaamini/Documents/ML_practice/ML_fundamentals/virtualenv/lib/python3.6/site-packages/scipy/optimize/linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/iaamini/Documents/ML_practice/ML_fundamentals/virtualenv/lib/python3.6/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


Epoque 307 and loss is: 3.5086340093613987e-06
Epoque 308 and loss is: 3.4827987267535084e-06
Epoque 309 and loss is: 3.4572482499043874e-06
Epoque 310 and loss is: 3.4319783813610344e-06
Epoque 311 and loss is: 3.406985005555217e-06
Epoque 312 and loss is: 3.3822640876933177e-06
Epoque 313 and loss is: 3.3578116724019784e-06
Epoque 314 and loss is: 3.333623881796455e-06
Epoque 315 and loss is: 3.3096969132825405e-06
Epoque 316 and loss is: 3.286027037225296e-06
Epoque 317 and loss is: 3.262610594306926e-06
Epoque 318 and loss is: 3.2394439927070406e-06
Epoque 319 and loss is: 3.2165237056159932e-06
Epoque 320 and loss is: 3.1938462684817664e-06
Epoque 321 and loss is: 3.1714082764345014e-06
Epoque 322 and loss is: 3.1492063823105468e-06
Epoque 323 and loss is: 3.1272372942101974e-06
Epoque 324 and loss is: 3.105497774187871e-06
Epoque 325 and loss is: 3.0839846364759683e-06
Epoque 326 and loss is: 3.062694746508084e-06
Epoque 327 and loss is: 3.041625019942211e-06
Epoque 328 and loss 

Epoque 596 and loss is: 8.205789389497864e-07
Epoque 597 and loss is: 8.176582348828755e-07
Epoque 598 and loss is: 8.147531149136889e-07
Epoque 599 and loss is: 8.11863468287253e-07
Epoque 600 and loss is: 8.089891851589646e-07
Epoque 601 and loss is: 8.061301566834097e-07
Epoque 602 and loss is: 8.032862749699536e-07
Epoque 603 and loss is: 8.004574331271499e-07
Epoque 604 and loss is: 7.976435252183329e-07
Epoque 605 and loss is: 7.948444461727984e-07
Epoque 606 and loss is: 7.920600919412366e-07
Epoque 607 and loss is: 7.892903593847093e-07
Epoque 608 and loss is: 7.865351461858314e-07
Epoque 609 and loss is: 7.83794351048613e-07
Epoque 610 and loss is: 7.810678734542086e-07
Epoque 611 and loss is: 7.783556138607587e-07
Epoque 612 and loss is: 7.756574735479589e-07
Epoque 613 and loss is: 7.729733546392638e-07
Epoque 614 and loss is: 7.703031601240914e-07
Epoque 615 and loss is: 7.676467938356202e-07
Epoque 616 and loss is: 7.650041605174009e-07
Epoque 617 and loss is: 7.6237516557

Epoque 897 and loss is: 3.474866195897373e-07
Epoque 898 and loss is: 3.466821099447885e-07
Epoque 899 and loss is: 3.4588039359255346e-07
Epoque 900 and loss is: 3.450814575656695e-07
Epoque 901 and loss is: 3.4428528905220484e-07
Epoque 902 and loss is: 3.434918752402271e-07
Epoque 903 and loss is: 3.427012034510305e-07
Epoque 904 and loss is: 3.419132610281134e-07
Epoque 905 and loss is: 3.411280354704051e-07
Epoque 906 and loss is: 3.4034551423242566e-07
Epoque 907 and loss is: 3.395656849019215e-07
Epoque 908 and loss is: 3.387885352220701e-07
Epoque 909 and loss is: 3.380140527584128e-07
Epoque 910 and loss is: 3.3724222545396667e-07
Epoque 911 and loss is: 3.3647304114072586e-07
Epoque 912 and loss is: 3.3570648771729804e-07
Epoque 913 and loss is: 3.349425532377215e-07
Epoque 914 and loss is: 3.3418122575603433e-07
Epoque 915 and loss is: 3.334224933928878e-07
Epoque 916 and loss is: 3.326663443799551e-07
Epoque 917 and loss is: 3.319127669933181e-07
Epoque 918 and loss is: 3.3

...

Epoque 21935 and loss is: 5.422037701816501e-10
Epoque 21936 and loss is: 5.421542542348592e-10
Epoque 21937 and loss is: 5.421048049014496e-10
Epoque 21938 and loss is: 5.420553555680402e-10
Epoque 21939 and loss is: 5.420058840301701e-10
Epoque 21940 and loss is: 5.419564346967605e-10
Epoque 21941 and loss is: 5.419070075678113e-10
Epoque 21942 and loss is: 5.418575582344017e-10
Epoque 21943 and loss is: 5.418081089009922e-10
Epoque 21944 and loss is: 5.417587039765034e-10
Epoque 21945 and loss is: 5.417092990520147e-10
Epoque 21946 and loss is: 5.416598719230654e-10
Epoque 21947 and loss is: 5.416104669985767e-10
Epoque 21948 and loss is: 5.415610620740879e-10
Epoque 21949 and loss is: 5.4151170155852e-10
Epoque 21950 and loss is: 5.414623188384917e-10
Epoque 21951 and loss is: 5.414129139140028e-10
Epoque 21952 and loss is: 5.413635978073559e-10
Epoque 21953 and loss is: 5.41314192882867e-10
Epoque 21954 and loss is: 5.412648323672991e-10
Epoque 21955 and loss is: 5.412154718517311

Epoque 22209 and loss is: 5.288962151522567e-10
Epoque 22210 and loss is: 5.2884854217568e-10
Epoque 22211 and loss is: 5.288008691991036e-10
Epoque 22212 and loss is: 5.287532628359084e-10
Epoque 22213 and loss is: 5.287056120637923e-10
Epoque 22214 and loss is: 5.286579390872156e-10
Epoque 22215 and loss is: 5.2861031051956e-10
Epoque 22216 and loss is: 5.285626597474437e-10
Epoque 22217 and loss is: 5.28515075588709e-10
Epoque 22218 and loss is: 5.284674470210533e-10
Epoque 22219 and loss is: 5.28419840657858e-10
Epoque 22220 and loss is: 5.283722120902024e-10
Epoque 22221 and loss is: 5.28324605727007e-10
Epoque 22222 and loss is: 5.282770437727326e-10
Epoque 22223 and loss is: 5.282294818184583e-10
Epoque 22224 and loss is: 5.281818976597234e-10
Epoque 22225 and loss is: 5.281343135009884e-10
Epoque 22226 and loss is: 5.280867737511744e-10
Epoque 22227 and loss is: 5.280392117969e-10
Epoque 22228 and loss is: 5.279916498426254e-10
Epoque 22229 and loss is: 5.279440656838905e-10
Ep

Epoque 22499 and loss is: 5.1533832703508e-10
Epoque 22500 and loss is: 5.152925192331785e-10
Epoque 22501 and loss is: 5.15246667022356e-10
Epoque 22502 and loss is: 5.152008370159939e-10
Epoque 22503 and loss is: 5.151550292140923e-10
Epoque 22504 and loss is: 5.151091770032699e-10
Epoque 22505 and loss is: 5.150633692013682e-10
Epoque 22506 and loss is: 5.150175613994665e-10
Epoque 22507 and loss is: 5.149717758020254e-10
Epoque 22508 and loss is: 5.149259457956633e-10
Epoque 22509 and loss is: 5.148801824026825e-10
Epoque 22510 and loss is: 5.148343746007807e-10
Epoque 22511 and loss is: 5.147886112078e-10
Epoque 22512 and loss is: 5.147428034058983e-10
Epoque 22513 and loss is: 5.146970622173781e-10
Epoque 22514 and loss is: 5.146513210288575e-10
Epoque 22515 and loss is: 5.146055576358768e-10
Epoque 22516 and loss is: 5.145598164473564e-10
Epoque 22517 and loss is: 5.145140530543755e-10
Epoque 22518 and loss is: 5.144683562747761e-10
Epoque 22519 and loss is: 5.144226150862557e-1

Epoque 22791 and loss is: 5.022070532157553e-10
Epoque 22792 and loss is: 5.021629329528452e-10
Epoque 22793 and loss is: 5.021188348943956e-10
Epoque 22794 and loss is: 5.020747368359462e-10
Epoque 22795 and loss is: 5.020306609819571e-10
Epoque 22796 and loss is: 5.019865851279681e-10
Epoque 22797 and loss is: 5.019425092739789e-10
Epoque 22798 and loss is: 5.018984334199899e-10
Epoque 22799 and loss is: 5.018543797704613e-10
Epoque 22800 and loss is: 5.018103261209326e-10
Epoque 22801 and loss is: 5.017662724714039e-10
Epoque 22802 and loss is: 5.017221966174147e-10
Epoque 22803 and loss is: 5.016781651723465e-10
Epoque 22804 and loss is: 5.016341337272782e-10
Epoque 22805 and loss is: 5.015901244866704e-10
Epoque 22806 and loss is: 5.015460708371416e-10
Epoque 22807 and loss is: 5.015020838009943e-10
Epoque 22808 and loss is: 5.01458052355926e-10
Epoque 22809 and loss is: 5.014140653197787e-10
Epoque 22810 and loss is: 5.013700560791708e-10
Epoque 22811 and loss is: 5.0132604683856

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

In [42]:
conjugate_gradient(w, loss, gradient, args=(X, y, 0.01))

Epoque 0 and loss is: 0.41990609969491144
Epoque 1 and loss is: 0.39971687123137634
Epoque 2 and loss is: 0.38752272024146417
Epoque 3 and loss is: 0.37965295914465
Epoque 4 and loss is: 0.3740182919162788
Epoque 5 and loss is: 0.36971820879100253
Epoque 6 and loss is: 0.3663186956857735
Epoque 7 and loss is: 0.3635782278316319
Epoque 8 and loss is: 0.36134252305712516
Epoque 9 and loss is: 0.3595023709953474
Epoque 10 and loss is: 0.3579757995008327
Epoque 11 and loss is: 0.35669960346325047
Epoque 12 and loss is: 0.35562451925758926
Epoque 13 and loss is: 0.35471196816382017
Epoque 14 and loss is: 0.3532265767858263
Epoque 15 and loss is: 0.3522508051792794
Epoque 16 and loss is: 0.3515297426968525
Epoque 17 and loss is: 0.3509640258462003
Epoque 18 and loss is: 0.35050386891935487
Epoque 19 and loss is: 0.3497747949275797
Epoque 20 and loss is: 0.34930622087550234
Epoque 21 and loss is: 0.34896266032513573
Epoque 22 and loss is: 0.34869378988472566
Epoque 23 and loss is: 0.348475081

In [272]:
import pandas as pd
from sklearn.model_selection import train_test_split

breast_cancer = pd.read_csv('../../DB/breast-cancer-wisconsin.data', sep=',', header=None)
breast_cancer.columns = ['col' + str(i) for i in range(len(breast_cancer.columns))]
breast_cancer.drop('col6', inplace=True, axis=1)
X = breast_cancer.iloc[:, 1:-1]
y = breast_cancer.iloc[:, -1]
y[y==2] = -1
y[y==4] = 1

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, 
                                                    test_size=.4, shuffle=True)

In [293]:
ionosphere = pd.read_csv('../../DB/ionosphere.data', sep=',', header=None)
X = ionosphere.iloc[:, 0:-1]
y = ionosphere.iloc[:, -1]

In [294]:
y = np.where(y == 'g', -1, 1)

In [295]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, 
                                                    test_size=.4, shuffle=True)

In [296]:
y_train

array([ 1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  1,  1, -1, -1, -1,  1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1, -1, -1,  1, -1, -1,  1,
        1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  1, -1, -1, -1, -1,  1, -1,
        1, -1,  1, -1, -1, -1, -1,  1,  1, -1, -1, -1, -1, -1,  1, -1, -1,
       -1,  1, -1,  1,  1, -1, -1, -1,  1, -1,  1,  1,  1, -1, -1,  1,  1,
        1, -1,  1,  1, -1,  1,  1, -1,  1,  1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1,  1,  1,  1, -1,  1, -1, -1, -1,  1, -1, -1,  1,  1, -1,  1,
       -1,  1, -1,  1, -1,  1, -1,  1, -1, -1,  1, -1, -1, -1,  1, -1, -1,
       -1,  1, -1, -1,  1, -1,  1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1,
       -1, -1,  1,  1,  1, -1, -1, -1, -1,  1, -1, -1, -1,  1, -1, -1,  1,
       -1,  1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1,  1,  1,
       -1,  1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  1, -1,  1, -1, -1, -1,
        1,  1,  1, -1, -1,  1])

In [299]:
sum(y == -1)

225

In [298]:
ionosphere.iloc[:, -1].value_counts()

g    225
b    126
Name: 34, dtype: int64

In [300]:
w_ex = np.random.random(X_train.shape[1]+1)

In [323]:
logistic_surrogate_loss(w_ex, X_train, y_train)

210
(210,)
(210,)


3.6242305798159125

In [313]:
loss(w_ex, X_train, y_train, 3)

3.413497346482656

In [202]:
conjugate_gradient(w=w_ex, cost_func=loss, 
                   grd_func=gradient, 
                   args=(X_train, y_train, 1))

In [172]:
w_ex

array([0.62100894, 0.35435185, 0.24700208, 0.6604293 , 0.5055178 ,
       0.23258424, 0.36056894, 0.656757  , 0.5053163 ])

In [199]:
fmin_cg(f=logistic_surrogate_loss, x0=w_ex, fprime=gradient_log_surrogate_loss, args=(X_train, y_train))

Optimization terminated successfully.
         Current function value: 0.115649
         Iterations: 83
         Function evaluations: 166
         Gradient evaluations: 166


array([ 0.56382516, -0.20256654,  0.59881419,  0.28872723,  0.20481866,
        0.64866464,  0.10685322,  0.59200928, -9.79636514])

In [198]:
fmin_cg(f=loss, x0=w_ex, fprime=gradient, args=(X_train, y_train, 0.01))

Optimization terminated successfully.
         Current function value: 0.115647
         Iterations: 141
         Function evaluations: 295
         Gradient evaluations: 295


array([ 0.56423199, -0.20289435,  0.59905301,  0.28893247,  0.20511434,
        0.64910797,  0.10676259,  0.5933518 , -9.8030808 ])